In [1]:
# Decision Networks

In [2]:
using BayesNets
using DataFrames

In [3]:
b = BayesNet()
push!(b, StaticCPD(:D, Bernoulli(0.01)))
push!(b, StaticCPD(:O1, [:D], Bernoulli(0.5))) # no real signal of whether disease is present
push!(b, FunctionalCPD{Bernoulli}(:O2, [:D], a->Bernoulli(a[:D] == true ? 0.9 : 0.01)))
push!(b, FunctionalCPD{Bernoulli}(:O3, [:D], a->Bernoulli(a[:D] == true ? 0.6 : 0.3)))

 overwritten in module DataFrames at U:\.julia\v0.5\DataFrames\src\abstractdataframe\abstractdataframe.jl:407.


In [4]:
U = DataFrame()
U[:T] = [false, false, true, true]
U[:D] = [false, true, false, true]
U[:U] = [0, -10, -1, -1]
U

4×3 DataFrames.DataFrame
│ Row │ T     │ D     │ U   │
├─────┼───────┼───────┼─────┤
│ 1   │ false │ false │ 0   │
│ 2   │ false │ true  │ -10 │
│ 3   │ true  │ false │ -1  │
│ 4   │ true  │ true  │ -1  │

In [5]:
function BayesNets.estimate(b::BayesNet, target::NodeName, consistent_with::Assignment; nsamples = 10000)
    srand(0)
    t = estimate(rand_table_weighted(b, nsamples=nsamples, consistent_with=consistent_with))
    normalize(sumout(t, setdiff(names(b), [target])))
end;

In [6]:
D = estimate(b, :D, Assignment(:O1=>true))

2×2 DataFrames.DataFrame
│ Row │ D │ p      │
├─────┼───┼────────┤
│ 1   │ 0 │ 0.9899 │
│ 2   │ 1 │ 0.0101 │

In [7]:
EU = join(U, D, on = :D)

4×4 DataFrames.DataFrame
│ Row │ T     │ D     │ U   │ p      │
├─────┼───────┼───────┼─────┼────────┤
│ 1   │ false │ false │ 0   │ 0.9899 │
│ 2   │ true  │ false │ -1  │ 0.9899 │
│ 3   │ false │ true  │ -10 │ 0.0101 │
│ 4   │ true  │ true  │ -1  │ 0.0101 │

In [8]:
by(EU, :T, df->dot(df[:U], df[:p]))

2×2 DataFrames.DataFrame
│ Row │ T     │ x1     │
├─────┼───────┼────────┤
│ 1   │ false │ -0.101 │
│ 2   │ true  │ -1.0   │

In [9]:
function diseaseEU(b::BayesNet, a::Assignment, U::DataFrame)
    D = estimate(b, :D, a)
    EU = join(U, D, on = :D)
    t = by(EU, :T, df->dot(df[:U], df[:p]))
    rename!(t, :x1, :EU)
    t
end;

In [10]:
diseaseEU(b, Assignment(:O1=>true), U)

2×2 DataFrames.DataFrame
│ Row │ T     │ EU     │
├─────┼───────┼────────┤
│ 1   │ false │ -0.101 │
│ 2   │ true  │ -1.0   │

In [11]:
diseaseEU(b, Assignment(:O1=>false), U)

2×2 DataFrames.DataFrame
│ Row │ T     │ EU     │
├─────┼───────┼────────┤
│ 1   │ false │ -0.101 │
│ 2   │ true  │ -1.0   │

In [12]:
diseaseEU(b, Assignment(:O2=>true), U)

2×2 DataFrames.DataFrame
│ Row │ T     │ EU       │
├─────┼───────┼──────────┤
│ 1   │ false │ -4.78698 │
│ 2   │ true  │ -1.0     │

In [13]:
t = diseaseEU(b, Assignment(:O3=>true), U)

2×2 DataFrames.DataFrame
│ Row │ T     │ EU       │
├─────┼───────┼──────────┤
│ 1   │ false │ -0.19998 │
│ 2   │ true  │ -1.0     │